### Setup Environment

In [34]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm synthetic-data-kit==0.0.3
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade         unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install synthetic-data-kit==0.0.3
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

In [35]:
pip install --upgrade uv

In [36]:
pip install unsloth vllm

  Using cached triton-3.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (1.5 kB)
Using cached triton-3.3.0-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (156.5 MB)
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0


In [37]:
#@title Colab Extra Install { display-mode: "form" }
import os
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    print("Not using Collab")
    
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade         unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
    !uv pip install synthetic-data-kit==0.0.3
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

Using Python 3.12.12 environment at: /usr
Audited 1 package in 89ms
Using Python 3.12.12 environment at: /usr
Resolved 18 packages in 19ms                                         
Uninstalled 1 package in 57ms
Installed 1 package in 43ms                                 
 - transformers==4.57.1
 + transformers==4.56.2
Using Python 3.12.12 environment at: /usr
Resolved 1 package in 1ms                                            
Uninstalled 1 package in 2ms
Installed 1 package in 5ms                                  
 - trl==0.23.0
 + trl==0.22.2


## Project Goal
In this project, we aim to fine-tune Llama 3.2 3B on GWU Computer Science course data.
We will use the `combined_courses.csv` file to generate question and answer pairs locally, which will then be used for fine-tuning.

In [38]:
from unsloth.dataprep import SyntheticDataKit

generator = SyntheticDataKit.from_pretrained(
    # Choose any model from https://huggingface.co/unsloth
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Longer sequence lengths will be slower!
)

INFO 11-22 19:12:46 [vllm_utils.py:700] Unsloth: Patching vLLM v1 graph capture
INFO 11-22 19:12:46 [vllm_utils.py:730] Unsloth: Patching vLLM v0 graph capture
Unsloth: Using dtype = torch.float16 for vLLM.
Unsloth: vLLM loading unsloth/Llama-3.2-3B-Instruct with actual GPU utilization = 71.65%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 4.58 GB. Also swap space = 0 GB.
Unsloth: `cudagraph_mode` is not in `from vllm.config import CompilationConfig`
Unsloth: Disabling `disable_cascade_attn` in vLLM to allow for better on policy RL!
vLLM STDOUT: INFO 11-22 19:13:01 [__init__.py:244] Automatically detected platform cuda.
vLLM STDOUT: INFO 11-22 19:13:04 [api_server.py:1395] vLLM API server version 0.9.2
vLLM STDOUT: INFO 11-22 19:13:04 [cli_args.py:325] non-default args: {'model': 'unsloth/Llama-3.2-3B-Instruct', 'dtype': 'float16', 

## Generate QA Pairs + Auto clean data
We now use synthetic data kit for question answer pair generation:

In [39]:
generator.prepare_qa_generation(
    output_folder = "data", # Output location of synthetic data
    temperature = 0.7, # Higher temp makes more diverse datases
    top_p = 0.95,
    overlap = 64, # Overlap portion during chunking
    max_generation_tokens = 512, # Can increase for longer QA pairs
)

In [40]:
!synthetic-data-kit system-check

vLLM STDOUT: INFO:     127.0.0.1:41090 - "GET /v1/models HTTP/1.1" 200 OK
 VLLM server is running at http://localhost:8000/v1
Available models: {'object': 'list', 'data': [{'id': 
'unsloth/Llama-3.2-3B-Instruct', 'object': 'model', 'created': 1763838885, 
'owned_by': 'vllm', 'root': 'unsloth/Llama-3.2-3B-Instruct', 'parent': None, 
'max_model_len': 2048, 'permission': [{'id': 
'modelperm-f8e0deea0840403aa569ca85bdac6d4d', 'object': 'model_permission', 
'created': 1763838885, 'allow_create_engine': False, 'allow_sampling': True, 
'allow_logprobs': True, 'allow_search_indices': False, 'allow_view': True, 
'allow_fine_tuning': False, 'organization': '*', 'group': None, 'is_blocking': 
False}]}]}
⠋ Checking VLLM server at http://localhost:8000/v1...


## Data Processing
We ingest the GWU Computer Science bulletin data to create Q&A pairs for fine-tuning.

In [41]:
# Ingest GWU Computer Science Bulletin data
!synthetic-data-kit \
    -c synthetic_data_kit_config.yaml \
    ingest "https://bulletin.gwu.edu/courses/csci/"

# Truncate document
filenames = generator.chunk_data("data/output/bulletin_gwu_edu.txt")
print(len(filenames), filenames[:3])

⠧ Processing https://bulletin.gwu.edu/courses/csci/.....
 Text successfully extracted to data/output/bulletin_gwu_edu.txt
16 ['data/output/bulletin_gwu_edu_0.txt', 'data/output/bulletin_gwu_edu_1.txt', 'data/output/bulletin_gwu_edu_2.txt']


We see around 37 chunks of data. We now call synthetic-data-kit to create some pairs of data for 3 of our chunks.

You can process more chunks, but it'll be much slower!

Using `--num-pairs` will generate **approximately** that many QA pairs. However it might be shorter or longer depending on the `max_seq_length` of the loaded up model. So if you specify 100, you might only get 10 since the model's max sequence length is capped.

In [42]:
import time
# Process 3 chunks for now -> can increase but slower!
for filename in filenames[:3]:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        create {filename} \
        --num-pairs 25 \
        --type "qa"
    time.sleep(2) # Sleep some time to leave some room for processing

vLLM STDOUT: INFO:     127.0.0.1:41102 - "GET /v1/models HTTP/1.1" 200 OK
vLLM STDOUT: INFO:     127.0.0.1:41116 - "GET /v1/models HTTP/1.1" 200 OK
⠏ Generating qa content from data/output/bulletin_gwu_edu_0.txt...vLLM STDOUT: INFO 11-22 19:14:48 [chat_utils.py:444] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.
vLLM STDOUT: INFO 11-22 19:14:48 [logger.py:43] Received request chatcmpl-afa10050de4245c5bbbcbf016027b0fc: prompt: '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Nov 2025\n\nSummarize this document in 3-5 sentences, focusing on the main topic and key concepts.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nComputer Science (CSCI) | The George Washington University\nSkip to Content\nAZ Index\nBulletin Home\nInstitution Home\nUndergraduate Admissions opens new window\nGraduate Admissions\nFees & Financial Regulations\nFinancial As

Optionally, you can clean up the data via pruning "bad" or low quality examples and convert the rest to JSON format for finetuning!

In [43]:
# !synthetic-data-kit \
#     -c synthetic_data_kit_config.yaml \
#     curate --threshold 0.0 \
#     "data/generated/arxiv_org_0_qa_pairs.json"

We now convert the generated datasets into QA formats so we can load it for finetuning:

In [44]:
qa_pairs_filenames = [
    f"data/generated/bulletin_gwu_edu_{i}_qa_pairs.json"
    for i in range(len(filenames[:3]))
]
for filename in qa_pairs_filenames:
    !synthetic-data-kit \
        -c synthetic_data_kit_config.yaml \
        save-as {filename} -f ft

⠋ Converting data/generated/bulletin_gwu_edu_0_qa_pairs.json to ft format with 
json storage...
 Converted to ft format and saved to 
data/final/bulletin_gwu_edu_0_qa_pairs_ft.json
⠋ Converting data/generated/bulletin_gwu_edu_1_qa_pairs.json to ft format with 
json storage...
 Converted to ft format and saved to 
data/final/bulletin_gwu_edu_1_qa_pairs_ft.json
⠋ Converting data/generated/bulletin_gwu_edu_2_qa_pairs.json to ft format with 
json storage...
 Converted to ft format and saved to 
data/final/bulletin_gwu_edu_2_qa_pairs_ft.json


Let's load up the data and see what the synthetic data looks like!

In [45]:
from datasets import Dataset
import pandas as pd
final_filenames = [
    f"data/final/bulletin_gwu_edu_{i}_qa_pairs_ft.json"
    for i in range(len(filenames[:3]))
]
conversations = pd.concat([
    pd.read_json(name) for name in final_filenames
]).reset_index(drop = True)

dataset = Dataset.from_pandas(conversations)

In [46]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the primary focus of courses in the 1000s?',
   'role': 'user'},
  {'content': 'Introductory undergraduate courses', 'role': 'assistant'}]}

In [47]:
dataset[1]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'Which courses in the 2000s to 4000s can also be taken for graduate credit?',
   'role': 'user'},
  {'content': 'Upper-level undergraduate courses', 'role': 'assistant'}]}

In [48]:
dataset[-1]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the primary focus of CSCI 3212?', 'role': 'user'},
  {'content': 'Core concepts in design and analysis of algorithms, data structures, and problem-solving techniques.',
   'role': 'assistant'}]}

Finally free vLLM process to save memory and to allow for finetuning!

In [49]:
generator.cleanup()

Attempting to terminate the VLLM server gracefully...
vLLM STDOUT: INFO 11-22 19:16:03 [launcher.py:80] Shutting down FastAPI HTTP server.
Server terminated gracefully.


### Fine-tuning Synthetic Dataset with Unsloth

In [50]:
from unsloth import FastLanguageModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",
    # Qwen3 new models
    "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # Other very popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [51]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the `Llama-3.2` format for conversation style finetunes. The chat template renders conversations like below: (Cutting Knowledge Date is by default there!)

```
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 01 May 2025

You are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is 1+1?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

2<|eot_id|>
```

In [52]:
def formatting_prompts_func(examples):
    convos = examples["messages"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }

# Get our previous dataset and format it:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

See result of the first row:

In [53]:
dataset[0]

{'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'What is the primary focus of courses in the 1000s?',
   'role': 'user'},
  {'content': 'Introductory undergraduate courses', 'role': 'assistant'}],
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 22 Nov 2025\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the primary focus of courses in the 1000s?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIntroductory undergraduate courses<|eot_id|>'}

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [54]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/34 [00:00<?, ? examples/s]

In [55]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
5.631 GB of memory reserved.


In [56]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 34 | Num Epochs = 12 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,4.166600
2,4.396400
3,4.498000
4,3.740700
5,3.298800
6,3.114200
7,2.507800
8,2.326500
9,2.027300
10,2.141900


In [57]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

89.7529 seconds used for training.
1.5 minutes used for training.
Peak reserved memory = 5.631 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 38.2 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! Use `apply_chat_template` with `add_generation_prompt` set to `True` for inference.

In [58]:
messages = [
    {"role": "user", "content": "What courses are available in Computer Science?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

CSCI 1011, CSCI 1012, CSCI 1020, CSCI 1021, CSCI 1022, CSCI 1023, CSCI 1024, CSCI 1025, CSCI 1026, CSCI 1027, CSCI 1028, CSCI 1029, CSCI 1030, CSCI 1031, CSCI 1032, CSCI 1033, CSCI 1034, CSCI 1035, CSCI 1036, CSCI 1021, CSCI 1022, CSCI 1023, CSCI 1024, CSCI 1025, CSCI 1026, CSCI 1027, CSCI 1028, CSCI 1029, CSCI 1030, CSCI 1031, CSCI 1032, CSCI 1033, CSCI 1034, CSCI 1035, CSCI 1036, CSCI 1021, CSCI 1022, CSCI 1023, CSCI 1024, CSCI 1025, CSCI 1026, CSCI 1027, CSCI 102


The model now answers questions about the GWU CS courses.

In [65]:
messages = [
    {"role": "user", "content": "Tell me about CSCI 6264 and who can take it."},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

CSCI 6264 is a graduate-level course focused on the design and analysis of algorithms. It is open to undergraduate and graduate students in the CSCI department, with approval from the instructor.<|eot_id|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [60]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [61]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
messages = [
    {"role": "user", "content": "What are the prerequisites for CSCI 2113?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer,
                   max_new_tokens = 256, temperature = 0.1)

CSCI 1112 and MATH 1220<|eot_id|>


### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [62]:
# Merge to 16bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # Change to True to upload finetune
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # Change to True to upload finetune
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False: # Change to True to upload finetune
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [63]:
# Save to 8bit Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
if False: # Change to True to upload finetune
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # Change to True to upload finetune
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # Change to True to upload finetune
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")